# Accessing data

The data sources we use come have varying levels of access restrictions, different ways of selecting (filtering) data, and use a few different file formats.

## Access restrictions

Most data sources are publically accessible, so we can just do whatever we need to download this data that exists in the public domain.

One data source that isn't public is Lightcast / EMSI Burning Glass. This dataset requires a paid account, and there doesn't appear to be a developer-friendly way of downloading the data programmatically. This means we'll have to login and download the data manually for now.

## Selecting / filtering data

Some of our data is available directly from a specific URL; however, much of our data is not directly accessible. This means we'll use these methods for accessing data - in order of simplicity:

* Simple HTTP request for a specific URL
* REST API (e.g. [CPS Public API](https://api.cps.edu/)); These kinds of APIs typically support filtering via URL query parameters and/or JSON request payloads.
* Stateless HTTP request with some specific payload (e.g. search criteria); It appears most of these payloads will use [webform encoding](https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods/POST#:~:text=%3Cbutton%3E%20elements%3A-,application/x%2Dwww%2Dform%2Durlencoded,-%3A%20the%20keys%20and).
* Stateful HTTP requests where our search criteria is stored on the provider's server; For these, we may be able to generate a session cookie and include this in a HTTP request; however, it's more likely the server state expires. This means we'd likely have to use (headless) browser automation if we want to automate the downloading of this data.

## File formats

Data is available in these forms - in order of preference:
* Tabular files - CSV, Excel spreadsheets, etc.
* JSON
* HTML
* PDFs 🤮

# Simple example: requesting a URL to download a CSV

In [4]:
pip install pandas xlrd

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 3.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
from urllib.request import Request, urlopen
from pandas import read_excel
    
url = "https://www.cps.edu/globalassets/cps-pages/about-cps/district-data/metrics/assessment-reports/assessment_psatsat_schoollevel_2021.xls"

req = Request(url)
req.add_header('User-Agent', 'P33') # CPS blocks requests from well-known development tools, so we need to spoof our user agent
with urlopen(req) as f:
    pd = read_excel(f.read(), sheet_name=2)
    
pd

,School Year,School ID,School Name,Test,# Students,Average Composite Score,# Students with EBRW Score,Average EBRW Score,# Students with Math Score,Average Math Score,% Meeting College Readiness Benchmark,# Students in Growth Cohort,Local Growth Percentile,# Students in EBRW National Growth Percentile,EBRW National Growth Percentile,# Students in Math National Growth Percentile,Math National Growth Percentile
0,2020-2021,400013,ASPIRA - EARLY COLLEGE HS,SAT,61,832,61,431,61,400,8.2,,,,,,
1,2020-2021,400013,ASPIRA - EARLY COLLEGE HS,Combined,61,,,,,,8.2,,,,,,
2,2020-2021,400022,CHIARTS HS,SAT,120,960,120,500,120,460,32.5,,,,,,
3,2020-2021,400022,CHIARTS HS,Combined,120,,,,,,32.5,,,,,,
4,2020-2021,400032,CICS - ELLISON HS,SAT,44,781,44,392,44,389,4.5,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,2017-2018,610581,BRIDGESCAPE - BRAINERD HS,SAT,18,796,18,413,18,382,0,,,5,,5,
1639,2017-2018,610581,BRIDGESCAPE - BRAINERD HS,Combined,18,,,,,,0,,,,,,
1640,2017-2018,610587,DYETT ARTS HS,PSAT 9,171,767,171,389,171,378,19.3,162,25,,,,
1641,2017-2018,610587,DYETT ARTS HS,PSAT 10,129,790,129,408,129,382,14,110,12,125,66,125,16


# Intermediate: stateless HTTP request with filtering

In [20]:
from urllib.request import Request, urlopen
from pandas import read_csv
    
url = "https://www.ibhe.org/EnrollmentsDegrees/EnrDegRace.aspx"

# This payload represents some ASP.NET view state - not super readable, but it contains the filter criteria...
payload = b"__EVENTTARGET=&__EVENTARGUMENT=&__VIEWSTATE=ixQqIICJWxYU3FahpfneVKwLBNCsU3EY8DMttkg7tgBucnX3rUDRS9EVcSr4U69MjHtsKnA68LBj3pL6j94WvqIZeuNB%2Bs2CCsuFs02pSqHG51SKKhfBmwrvLDUhblLyshBYv5pWRGXJCyDO%2Bo0rlBF9XeKQKupfn%2Fc%2F8JtsWDm40utRcjftN9Gqwn6GNG%2FfgaMToApXvl594UrlXTHEZrwFjBP1ldCQHOArF1h6pX9x0%2FrSnzyGeGVlDLSEd%2F3tjbQhsS22kM5UNXd%2By%2BNXSiYD0e5UxiLj%2B6S28XYDPHpFBqFsEznPZ%2BHUq9hJ6NqhzTW4RgG9RqvR4FV6mgTXZQ26kgPPYbPQR2QYpk56ntHwJKZDCWjGp6hZYpFEtcGdhX%2BsVCPXq3W%2F9oXImLJZM5CSmv1C9HIqe1q4P89aPLWBtMrc4wK8CifLtxLxMnhMR3l6AsMXUJpuLVPYjB%2F1922kLKjy6%2BCiZvz3VMFzCl5BKXT6ZzerPjrvtZw3gtP8odCiHSgwbdOpypkm3fPw0cYWVTYl%2Fb75W8JYiJfwWSdAsq3FJOLFe%2F3G%2F%2BDUdoLqFNQGjtPEwdklTWppb3VkvwnfSUfQaVtr8Vnirzl%2FRRB3z4BQNeQ7B1TdZEhQnh9PHbSo8I3dJHJwbkyfkyA7jyqH6U%2FjhRH%2Fts5IKOnXmjBsQ7Jy25QRSnDNzRqpVxpwB1cAPnrD1sPCnz3mEAIhU48gZdUipT5OxhejW2aKj9d7rN%2BQeqkN6iKs7r1GAyBPEuiWVEUdHQP8HXRNdfKV4iXKk2zvXdgkeGK5rHOD0htvmPh4veIt4y%2Fvo5iQSh5ZkW7plRQnVOBdRoNNeDP4SQ2ECreAS3S%2FL5PORNsX2o8dURP1QVSKHSlEY2h4LtWWDIqdG0zSWGQ0rOYpaOK%2FLVg1BrVqGiW10ZMfw7iIgDAHj4GnTh58ThB%2B3sg39u9NZtb8N%2B2Tcv8qpJ8oGDR4rg393HEVeJT%2Fronf1pV7p9X2B5t7vH4mFbduFiBl3UerChlYo94A9jTZYFbbMOfituW0wR5wqrSagJ9nf1itOuiFTPoBPPpW0xESSYNXnZ6JC6YSC3QF0IxDbAEghpSo7METOD8eS5YO4HJx%2BWIJCjekQv0W4YtriFjJ21yMIhay2h%2By8Qe0yaZphb%2FrQrKAh1L7COpHpsPIJgswtzYCRlVUPLFKErGlMVTYiWL%2FsLAHd6n2Ol4sxZfXmK53jGuPLp58Ro9IYs8ElMHLj%2FfEnQ7iEmc7vYqAlrWkv6meY%2FYPad9jEmsKfqYngReyICZL%2B7ZgP3B%2FTU%2FTZU9WBJOsr7IgXi5k3eot4uAJO1nTRXXSt4UaldqPv9fFiZDZLilBMAp3QI5mkTuc2ts1fi9wJXWxr45jr65HVU3yc%2FLyXj%2FM9GbYBi0e32j4K7Q5h5936mCobbRF%2F%2BDiz6QVX2TAixLpwaJbKFwPhEKwOToC5cxq32v0uhQqr1l%2BkjPo7xa8%2BJv5M0vsL5iOZaNKaSW%2F3s0J8vZz8OAVYeodY5YdK1UfuO9OS58dIpvByElU8bfjuqI0SouIeS4Q6VO7xVMBPek3SQIuDbdqJ9M1oMOnMuoRc%2B3QGRgaLlKXx5Bj4az1TDrrZNoIETJDNJ2oZEGz6DMv2YXTL19f0LGQPqzR3fQ1nz77Qg28JzJB9SvCYc7B52%2FSDfFhcHpZCH9%2FnhCv7L7yUXiyZ3L97oj7dQJSHetR%2BjtOCWzc4VySPlxi8MaNFhylVL6M%2F7M3FlwEfN7D87BjKsh8a06oTLdzxX%2F3PDIQqOBy3%2F5VvFwlES16%2BsAUrDzn27JLwgHEAN%2Bm%2BG82gffDfoNY3NyI0jVVr7ijvx1NFw8M5ZAUCNeyiMkO7dVMWjEWSmkiuF2FLcFoZEx7lnsSarqP89oBNbfy4Hhnf0xCLs%2B7F2FK7zgDqwGgQYyjXavYvLozqpKXOpHndmUO38ww%2B2MaPFbjy92%2B5ARkXcjoeSd0D5Fz14bEDFc%2BK3KpckSfznPw9VqjbvYFLrjUICGUCLENCP0fzxfO%2BITw3SKEKaoS%2FOqOdN84mAcC6oxtueHgTQ1XmQQkqVZWsy0FQbqZ9M015VztMIiwpVDteh9s654zcUSwf0z5y68eY6GccCvnOprMJP6Zfk5MBwe4%2FVdLc2lZCgvSoo7jVSqf%2Bvq313ifM6dJx%2BY6TNbNDxVHoyskFUVZkI3XCDHwzrr%2ByaDSoVETfopJ2ciRTeVd%2F5qNWQRjMKizwKwKlMOckOctXjHQNa36o2hcgk4WJrojIkXoz0UfhxHQRUCDw1QQY2wcAgLmHn%2Bwm3y%2Fq7O05Jxd8P5AjZE65ySQE22%2B41whL0JBihZDJwdbUZAhMrjtTF4QxR6o52WPw26ZWZqQJUgQ5S7ANvJuXLrYflsCOBot3WsXUe7P%2F6jk6iR0E8IsmjypaP7FmRYRHI0ZzmxPq55mnPF2YDuvGj7Rol5URxvocsJhOy5elpFT%2FDxwMWl7X7Z1DziyXN4IqPQpDDPgCYdGMKs4VwnLIxZuMpMYxz321429%2FrjzZ4W57ZKzUZFKZYsUNyWw7UQXWcoOtN%2BLCXnJWesy3qoOa%2BUN4vPW%2BlWDuP3NJYZRp%2Bo03lcU%2FY2LwzBHxMMUqR2Api6XEF28%2FHMXmaeoBaRfhTGN%2B1Sn6jIfwBU8bxbr34miZyyADuZWcXmeMvPvmMEwMN8ptpUy1EuEOKLyFonVfJlEu5eqc42RXtJl%2B7%2FI87dido7ROMYIhfTjPJ27ROqUuRv1joN2ocxyNqhf8H4LrHISNoy3FSVabep3af6F07WRLqOQqELgL90AVvCAawHsbhK8JvVpCrDIxqO7ZVdGIbwUP6ZjfjiWQZ0GDLxu7ZFFNaPEoiIz2mv879HYGtXqaMAZrdYo6H0Yx8N0ChT42HOJfMkAYsThQHbTvjNW6o%2B2U38%2Bk8S%2Fpp3%2Fj7PUd72CNsVGHvX1dma4pRVYJ%2B9alDK6BjktxBTY5vv6xXpeVVQ71rSWYdgo3P4PPHJnWoL04m4OhYSA2%2BkYjcyJrm5jfe2%2Fb%2ByCxG08C0XzB8ll%2FMQwM8mRcmb8yZ3woyBl%2BPFhxIKzw%2FzazX%2FTshRWw8JYFO7hbgObJZXwHmOyGLfDmQVg8ibpZrJrug%2FFLyV1rSLScKsUa2b2wB6loFJNgnnqaLMC0ZcHigfTqJ%2BfacMwrjJQTZt6yKNPMZX1z5aSkaUYjULcnJHDKRgPwWXTjUaYgUaBpwnxhwR5pFyP3M%2Fw5xOrezz14u2Mi27S%2B6PxZcvNV2%2BwQYvH7npHnw%3D%3D&__VIEWSTATEGENERATOR=000C3DC2&__EVENTVALIDATION=mp9h%2F%2B%2Fl0%2FKfuwAXZBavSlaF%2FmciPXHgMmZA4v5qXpWfS20J%2BBfKxdrGQX4WQPn%2FcKEeNBtjGfLdFxLv6boLKo8WPGpdsyIt7QTdnQTa4Tx482mY9%2FYg4%2FLmVgklwKTYYvontE%2Bd95upSpxFxozDQociEnzwEcir7TX8aB5lMPjO1sgHKLMWLD%2F9qjiIYLM4ltyzu7gBlDWA%2BCLKXAf8yA%3D%3D&ctl00%24ContentPlaceHolder1%24cboExport=2&ctl00%24ContentPlaceHolder1%24btnExcelExp2=Export"

req = Request(url)
req.data = payload
with urlopen(req) as f:
    pd = read_csv(f)
    
pd

,Year,Sector Code,Unnamed: 2,"Afr-Amer, Black","Amer Indian, AK Native",Asian,Hispanic,"Native Hawaii, Other Pacific",White,2+ races,Intl. Stud.,Unknown,Total,Unnamed: 13
0,2021,1.0,Public Universities,20544,179,19527,28610,117,85543,5436,19965,4837,184758,NaN
1,2021,2.0,Community Colleges,27439,609,11662,62994,184,111254,6760,1652,7936,230490,NaN
2,2021,3.0,Independent NFP Institutions,17919,332,18327,34441,198,98025,5695,19962,9016,203915,NaN
3,2021,5.0,Independent For-Profit Institutions,12144,334,3171,8259,282,25092,985,634,3139,54040,NaN
4,2021,7.0,Out-of-State Institutions,1621,34,276,360,13,2100,81,66,1079,5630,NaN
5,2021,NaN,2021 Year Total,79667,1488,52963,134664,794,322014,18957,42279,26007,678833,NaN


# Advanced: stateful web form

We can use [scrapy](https://scrapy.org/).

# Advanced: private access with web form filtering

# Writing to filesystem

In [28]:
from os.path import basename
from os import remove
from urllib.parse import urlparse
from urllib.request import Request, urlopen
    
url = "https://www.cps.edu/globalassets/cps-pages/about-cps/district-data/metrics/assessment-reports/assessment_psatsat_schoollevel_2021.xls"
filename = basename(urlparse(url).path)

req = Request(url)
req.add_header('User-Agent', 'P33') # CPS blocks requests from well-known development tools, so we need to spoof our user agent
with urlopen(req) as response:
    with open(filename, 'wb') as out:
        out.write(response.read())

# comment out the next line if you want to view the file
remove(filename)